In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

file_path = r"F:\guvi\Capstone Projects\IMDB-project\data\csv"
base_url = r'https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31'
genres = ['action','documentary','adventure','animation','drama','romance','fantasy','history','mystery','talk-show', 'news', 'reality-tv'] 
driver = webdriver.Chrome()
for genre in genres:
    driver.get(base_url+ "&genres="+genre)
    movies =  []
    is_more_items = True
    i = 0
    while is_more_items:
        time.sleep(3)
        cnt = 50 * i
        
        list_items= driver.find_elements(By.CSS_SELECTOR,"li[class='ipc-metadata-list-summary-item']")
        print(f"genres={genre}, cnf={cnt} listitems= {len(list_items)}")
        for item in list_items[cnt:]:
            title_ele = item.find_elements(By.CSS_SELECTOR,"h3[class='ipc-title__text ipc-title__text--reduced']")
            title = title_ele[0].text
            print(title)
            metadata = item.find_elements(By.CLASS_NAME, "dli-title-metadata-item")

            year = metadata[0].text if len(metadata) > 0 else '0'
            duration = metadata[1].text if len(metadata) > 1 else '0h'
            certificate = metadata[2].text if len(metadata) > 2 else ''
            rating_ele = item.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")
            rating = float(rating_ele[0].text) if len(rating_ele) > 0 else 0
            voting_ele = item.find_elements(By.CLASS_NAME, "ipc-rating-star--voteCount")
            voting_txt = voting_ele[0].text if len(voting_ele)>0 else "0"
            voting_txt = voting_txt.replace(" ","").replace("(","").replace(")","").replace("K","")
            voting = int(float(voting_txt)*1000 )

            rec = [genre.title(),year,title,duration,certificate,rating, voting]
            movies.append(rec)
        # trying to find the see more button to get the next 50 items    
        try:
            seemore = driver.find_element(by=By.XPATH, value=".//span[@class='ipc-see-more__text']")
            print(seemore)
            ActionChains(driver).move_to_element(seemore).perform()

            btn_element = seemore.find_element(by=By.XPATH, value=".//ancestor::button")
            ActionChains(driver).move_to_element(btn_element).perform()

            btn_element.click()
            time.sleep(3)
            is_more_items = True
            i += 1
        except NoSuchElementException:
            print("No more items")
            is_more_items = False

    df = pd.DataFrame(movies, columns=['genre','year','title','duration','certificate','rating', 'voting' ])
    print(df.head())
    
    df.to_csv(path_or_buf=f"{file_path}/{genre}.csv", mode="w")

driver.quit()